# DIC Crack Inspector

In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from io import BytesIO
import importlib
import bmcs_shear.dic_crack.dic_test_report as reporter

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
from bmcs_shear.dic_crack import sz_tests_series_2023 as ts
import numpy as np
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [ ]:
tests = [ts.B1_TV1, ts.B1_TV2, ts.B6_TV1, ts.B6_TV2, ts.B7_TV1, ts.B7_TV2, 
         ts.B8_TV1, ts.B8_TV2, ts.B9_TV1, ts.B9_TV2, ts.B10_TV1, ts.B10_TV2]

In [ ]:
def fig_test_subplots():
    fig = plt.figure(figsize=(10, 3))
    fig.canvas.header_visible = False
    grid = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=[0.4])
    ax_list = fig.add_subplot(grid[0, :])
    return fig, ax_list

def fig_crack_detection_subplots():
    fig = plt.figure(figsize=(10, 3))
    fig.canvas.header_visible = False
    grid = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=[0.4])
    ax_list = [fig.add_subplot(grid[0, :2]),
               fig.add_subplot(grid[0, 2])]
    return fig, ax_list

def fig_crack_t_subplots():
    fig = plt.figure(figsize=(10, 3.5))
    fig.canvas.header_visible = False
    grid = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=[1])
    ax_list = [fig.add_subplot(grid[0, i]) for i in range(3)]
    return fig, ax_list

def fig_cor_t_subplots():
    fig, ax = plt.subplots(1,1, figsize=(10, 3))
    fig.canvas.header_visible = False
    fig.tight_layout()
    return fig, ax


In [ ]:
active_tests = tests #  [tests[0]]
active_tests = [ts.B10_TV1]
dcl_tests = [ts.new_dcl(test) for test in active_tests]

In [ ]:
def plot_crack_detection_field(dcl, ax, fig, t_detect=None):
    if t_detect is None:
        t_detect = dcl.t_detect
    dcl.dsf.plot_crack_detection_field(ax, fig)
    ax.set_aspect("equal")

Set up the reporter

In [ ]:
importlib.reload(reporter)
report = reporter.DICReportTemplate(
    output_dir='.')

In [ ]:
importlib.reload(reporter)

n_tests = len(dcl_tests)
figs = []
specimen_names = []

ts_test_setup = (r'''
    \subsection{Design parameters}
    ''' )

ts_DIC_load_history = (r'''
    \subsection{DIC import and load history parameters}
    ''')

ts_interpolator = (r'''
    \subsection{Interpolator parameters}
    Parameters of the base grid constructed from unstructured data. This grid serves for the 
    evaluation of strain field and the fields derived based on constitutive assumptions, i.e.
    damage, stress, and energy release fields.  
    ''')

ts_crack_detection = (r'''
    \subsection{Crack detection parameters}
    ''' )

ts_crack_kinematics = r'''
    \subsection{Crack kinematics}

    The three diagrams show the values of maximum compressive strain at the top of the section, 
    the maximum crack opening and sliding at the bottom of the section. The shape of the curves can be 
    used as an indicator of the initiating failure mechanism.  
    '''
ts_tooth_kinematics = r'''
    \subsection{Tooth kinematics}
    The left diagram shows the evaluated centers of rotation are plotted for the crack detection load level 
    $t_\mathrm{detect} \in (0,1)$. This load level marks the end of the primary crack propagation that appears to comply 
    with the rotational tooth kinematics around a crack. Once the a dowel failure appears 
    in one of the cracks, a unique center of rotation cannot be found.  
    '''
ts_M_phi = r'''
    The following diagram uses the above displayed centers of rotation evaluated for the whole loading history 
    as a basis for the quantification of the relation between the bending 
    moment and rotation between two neighboring teeth. Note that the maximum value of the 
    moment decreases from the left to the right corresponding to the linear profile 
    of the bending moment along the length $a$.    
    '''

for dcl in dcl_tests:
    report.add_section(dcl.dir_name)
    fig, ax = fig_test_subplots()
    fig_crack_detection, (ax_cracks, ax_ld) = fig_crack_detection_subplots()
    fig_cr, (ax_cr_eps, ax_cr_w, ax_cr_s) = fig_crack_t_subplots()
    fig_cor, ax_cor_loc = fig_cor_t_subplots()
    fig_M_phi, ax_M_phi = plt.subplots(1,1, figsize=(6,4))
    # dcl.bd.plot_sz_bd(ax)
    plot_crack_detection_field(dcl, ax, fig)
    dcl.dsf.dic_grid.sz_bd.plot_sz_bd(ax)
    # dcl.plot_primary_cracks(ax_cracks=ax)
    ax.set_title(f'FE-DIC damage at $F_{{\max}}$',
        loc='left')
    ax.set_aspect("equal")
    ax.title.set_position([.2, 0.6])

    dcl.dsf.plot_crack_detection_field(ax_cracks, fig)
    dcl.plot_primary_cracks(ax_cracks=ax_cracks)
    ax_cracks.set_title(f'Cracks at detection state $t$={dcl.t_detect}$F_{{\max}}$ extended to ultimate state', loc='left')
    ax.title.set_position([0,1])
    ax_cracks.set_aspect("equal")

    dcl.dsf.dic_grid.dic_inp.plot_load_deflection(ax_ld)
    ax_ld.set_title('Load-deflection')

    M_Ct, phi_Ct = np.einsum('CMt->MCt', dcl.M_phi_Ct)
    phi_Ct = np.rad2deg(phi_Ct)

    F_T = dcl.dsf.dic_grid.dic_inp.F_T
    t_T = dcl.dsf.dic_grid.t_T
    for C, (cr, M_t, phi_t) in enumerate(zip(dcl.cracks, M_Ct, phi_Ct)):
        u_Ta = np.max(cr.get_u_crc_Ka(t_T, cr.X_crc_1_Ka[:5,:]), axis=1)
        ax_cr_w.plot(u_Ta[:, 0], F_T, label=f'crack {cr.name}', color=cr.color)

        eps_Tab = np.min(cr.get_eps_Kab(t_T, cr.X_1_Ka[-30:,:]), axis=1)
        eps_T00 = eps_Tab[:, 0, 0]
        ax_cr_eps.plot(eps_T00, F_T, color=cr.color)

        ax_cr_s.plot(u_Ta[:, 1], F_T, label=f'crack {C+1}', color=cr.color)

        ax_M_phi.plot(phi_t, M_t, label=f'crack {cr.name}', color=cr.color)

    ax_cr_eps.set_ylabel(r'$F$/kN')
    ax_cr_eps.set_xlabel(r'$\varepsilon_{\mathrm{top}}$/-')
    ax_cr_eps.set_title('Load vs. max. compression strain')
    ax_cr_eps.grid()

    ax_cr_w.set_ylabel(r'$F$/kN')
    ax_cr_w.set_xlabel(r'$w_{\mathrm{bottom}}$/mm')
    ax_cr_w.set_title('Load vs. max. crack opening')
    ax_cr_w.grid()

    ax_cr_s.set_ylabel(r'$F$/kN')
    ax_cr_s.set_xlabel(r'$s_{\mathrm{bottom}}$/mm')
    ax_cr_s.set_title('Load vs. max. slip')
    ax_cr_s.grid()
    ax_cr_s.legend()

    dcl.dsf.dic_grid.t = dcl.t_detect
    dcl.dsf.dic_grid.plot_bounding_box(ax_cor_loc)
    for C, cr in enumerate(dcl.cracks):
        print('crack', C)
        cr.cor._plot_crack_cor(ax_cor_loc)
        cr.plot_X_t_Ka(ax_cor_loc)
    ax_cor_loc.set_axis_off()
    ax_cor_loc.set_aspect('equal')
    ax_cor_loc.set_title(f'Center of rotation at $F$ = {dcl.t_detect}$F_{{\max}}$')

    ax_M_phi.set_ylabel(r'$M$/kNm')
    ax_M_phi.set_xlabel(r'$\phi$/deg')
    ax_M_phi.set_title('Bending moment vs. tooth rotation')
    ax_M_phi.grid()

    fig.subplots_adjust(wspace=0.3, hspace=0.4)
    fig_crack_detection.subplots_adjust(wspace=0.3, hspace=0.4)
    fig_cr.subplots_adjust(wspace=0.3, hspace=0.4)
    fig_cor.subplots_adjust(wspace=0.3, hspace=0.4)

    plt.close(fig)
    plt.close(fig_crack_detection)
    plt.close(fig_cr)
    plt.close(fig_cor)
    plt.close(fig_M_phi)

    report.add_text(ts_test_setup
                    + dcl.dsf.dic_grid.dic_inp.get_latex_design_params())
    report.add_figure(fig)
    report.add_text(ts_DIC_load_history
                    + '''The relative position of the DIC window and the applied
                    padding with respect to the boundary of the exported point cloud
                    is given in the following table.''' 
                    + dcl.dsf.dic_grid.dic_inp.get_latex_dic_params())
    report.add_text(ts_interpolator 
                    + dcl.dsf.dic_grid.get_latex_grid_params()) 
    report.add_text(ts_crack_detection 
                    + '''The parameter values applied in the crack detection algorithm 
                    of the current test are summarized in the following table.
                    '''
                    + dcl.get_latex_detection_params())
    report.add_figure(fig_crack_detection)
    report.add_text(ts_crack_kinematics)
    report.add_figure(fig_cr)
    report.add_text(ts_tooth_kinematics)
    report.add_figure(fig_cor)
    report.add_text(ts_M_phi)
    report.add_figure(fig_M_phi, width=0.5)

    report.add_text(r'''
        \section{Breakdown of stress transfer mechanisms versus equilibrium} 
    ''')
    for C, cr in enumerate(dcl.cracks):
        fig_ST, (ax_N, ax_V, ax_M) = plt.subplots(1,3, figsize=(10,3))
        sp = cr.sp
        sp.plot_ST(ax_N, ax_V, ax_M)
        report.add_figure(fig_ST, caption=f'''
                        Stress transfer mechanisms in crack {C+1} 
        ''')
        plt.close(fig_ST)

report.generate_report()

In [ ]:
dcl.cracks[4].X_crc_t_Ka